## Final Project: Lane Tracking and Keeping System for an Autonomous Vehicle


Self-driving vehicle with lane keeping system functionality. The vehicle uses a Kalman Filter for tracking lanes and a single monocular camera for detection purposes. The target platform is a Raspberry Pi 3B+. 

Written by: Enrique Mireles Gutiérrez  
ID Number: 513944  
Bachelor: ITR  
Date: 2019-05-23


### Introduction

Lane tracking and keeping systems have started to appear on commercial vehicles lately. Even though its development started long ago, it has only been a couple of years since its application has become commercially available. This technology not only detects lanes but also makes sure that the vehicle remains centered on the lane while driving. This is presents quite a different scenario to commonly known lane departure systems, which alert the driver when its leaning towards an edge on the lanes. 

Solving the problem of keeping a vehicle centered within lanes is no easy task. Therefore, the problem must be broken down into smaller problems that can be easily tackled by themselves. Two key components of the system are lane tracking systems and lane keeping systems. Furthermore, lane tracking systems consist on lane detection systems and filters such as Particle filters and Kalman filters.

On previous lab reports, the problem of detecting lanes was thoroughly solved. However, this final project required to develop filters for tracking the lanes and control strategies for keeping the vehicle centered while moving forward. For tracking purposes, a Kalman Filter was used and for control purposes a PID control loop was implemented.


### Objectives

The main objective is to develop an autonomous vehicle capable of driving around a course as fast as possible without leaving its lane. The course is going be a track with a total length of 400m. 

### 1. Basis

The Raspberry Pi 3B+ (RPi for short) is the brains on the vehicle. It will run the lane detection algorithms as well as the control algorithms for the motors and a web server for live monitoring and control of the vehicle. A STM32 microcontroller will aid the the RPi through serial communication by interfacing with the H-Bridge, reading the batteries' voltages, and providing some basic user feedback.

The reasons behind using a STM32 microcontroller for the reasons mentioned before are varied. The main one relies on moving all IO operations outside of the Raspberry Pi to focus its work on the lane detection algorithms and web server. Furthermore, it adds a layer of protection between the RPi and the outside world. Finally, since the STM32 microcontroller runs at 3.3V its the perfect match for the RPi.

#### 1.1 Prerequisites

- Python 3.6.7
- OpenCV 3.4.3 (lane detection).
- FilterPy 1.4.5 (for implementing the Kalman Filter).
- Numpy 1.16.2
- Imutils (for a threaded RPi camera implementation).

#### 1.2 Network

The main idea is to create a network hotspot on a laptop to which the Raspberry Pi is going to connect. The Raspberry Pi will be running its server locally and will try to connect automatically to the network every time it is in range. Doing so allows for a constant server operation and whenever a connection is lost or the hotspot isn't running, the server keeps on running looking for any connections.

Additionally, the created hotspot will be a bridged hotspot between any available wireless networks connected on the laptop. This means that whenever the RPi connects to the hotspot it will automatically gain internet access to the Internet. This implies that a second wireless card must be used. In my case, the internal wifi card of the laptop was used for Internet access and a external Etekcity 300 Mbps 2.4Ghz SI-7200ND wireless adapter was used for the hotspot creation.

The hotspot's network details are:
- Name: `mini-tesla`
- Passworkd: `elonmusk`
- IP: `10.0.0.1`
- Subnet mask: `255.255.255.0`
- Gateway: `blank` (this causes the hotspot to route all trafic to the other available gateway which connects to the Internet.)

The RPi network address is:
- IP: `10.0.0.2`
- Subnet mask: `255.255.255.0`
- Router: `10.0.0.1`

#### 1.2.1 Creating a Hotspot

The next instructions should be followed on the computer where the hotspot will be created. These instructions were executed on a Laptop running Ubuntu 18.04 on XFCE.

1. Go the Wi-Fi icon on the status bar.
2. Select the option `Edit Connections...` and press the `+` button.
3. Connection type will be `Wi-Fi`.
4. Configure the network as follows:
	- Wi-Fi > Connection name: `mini-tesla-hotspot`
	- Wi-Fi > SSID: `mini-tesla`
	- Wi-Fi > Mode: `Hotspot`
	- Wi-Fi > Device: depends on your setup
	- Wi-Fi Security > Security: `WPA & WPA2 Personal`
	- Wi-Fi Security > Key: `elonmusk`
	- IPv4 Settings > Address > Add:
		- Address: `10.0.0.1`
		- Netmask: `24`
		- Gateway: (do not fill, leave blank)
5. Once the network is configured go again to the Wi-Fi icon and select `Connect to Hidden Wi-Fi Network...`.
6. Finally, select the corresponding Wi-Fi adapter and the newly created connection. Then press connect.


#### 1.2.2 Static IP for the RPi

The following commands will set a static IP address on the RPi. Important to note is that by doing so you may loose access to other networks, since no automatic IP assignment will be made. However, ethernet connections should still work as normal.

1. Edit `/etc/dhcpcd.conf` and write at the end of the file:
        
        interface wlan0
        static ip_address=10.0.0.2/24
        static routers=10.0.0.1
        static domain_name_servers=10.0.0.1

2. Connect manually to the hotspot `mini-tesla` on the RPi.

Once the RPi has successfully connected to the hotspot, a SSH connection can be established by executing the following command `ssh pi@10.0.0.2`. Also, to test that the RPi has a connection to the Internet through the hotspot, make sure that the hotspot computer has access to the Internet and then execute a ping on the RPi to a known website (`ping google.com`).

#### 1.3 OpenCV 3.4.3

OpenCV 3.4.3 was the library used for image processing. It was compiled from source following these tutorials:
- https://www.alatortsev.com/2018/09/05/installing-opencv-3-4-3-on-raspberry-pi-3-b/
- https://www.alatortsev.com/2018/11/21/installing-opencv-4-0-on-raspberry-pi-3-b/

As one can see the second tutorial is for OpenCV 4.0. Nonetheless, these tutorial provides some useful tips for compiling OpenCV from source. These include: 
- Increasing swap memory temporarily.
- Installing `screen` to keep processes running when a SHH session is dropped.

#### 1.4 Arduino and STM32

Since the STM32 microcontroller will monitor and control key aspects of the vehicle it becomes important for the RPi to be able to communicate to the device and to program it.
Since the STM32 has a bootloader for UART programming, only two wires are required between the RPi for both tasks: TX and RX.

By following these tutorials it was possible to install the latest Arduino IDE and the required files for compiling programs for the STM32 platform. Some key aspects of these section involved assigning the high speed UART port on the RPi to the TX and RX terminals on its header, compiling the stm32flash program, and installing the STM32 board and libraries for Arduino.

- https://www.raspberrypi.org/magpi/program-arduino-uno-raspberry-pi/
- https://siliconjunction.wordpress.com/2017/03/21/flashing-the-stm32f-board-using-a-raspberry-pi-3/
- https://github.com/rogerclarkmelbourne/Arduino_STM32/wiki/Installation


### 2. Project Structure

The `main` folder contains the required files for running the lane keeping system on the RPi:
- `car.py`: consists of the interface between `main.py` and the STM-32 controller. When started, it spwans a thread that monitors constantly the serial port for messages. The STM-32 controls the motors' speed and monitors the voltage of the batteries.
- `processor.py`: implements the lane detection algorithms. Output from this class are the detected lanes.
- `tracker.py`: implements the Kalman Filter for processing the output of the Processor class. Outputs the filtered lanes.
- `line.py`: consists of a class for storing the points found via the Hough-Transform. Outputs the first order polynomial that best fits those points.
- `main.py`: entry point for the program. Takes the output from the camera, passes the frame to the processor class, calculates the pid output, and sends the control speed via the car class.

### 3. Working Principle

The main working principle behind the lane keeping system relies on the idea that for a system that estimates the lane position via a first order polynomial, the position of the lanes will always be the same when the vehicle's position is centered on the lanes. This true regardless on the lane curvature. Since the vehicle is moving at a slow speed, such an estimate can be made and produces reliable results. 

The following image shows in white the center position of the lanes, and in yellow and cyan the currently detect lanes. Since the vehicle is relatively centered, the position of the detected and centered lanes is about the same.
<img src="tracking-center.png" alt="Vehicle centered on the lanes." width="500">

In the following image, the vehicle turned suddenly to the right. Therefore, the detected lanes are to the left from the centered lines. By measuring the -x distance from the tip of the centered lanes to the tip of the detected lanes, an indication of how much the vehicle has departed from the center can be estimated.
<img src="tracking-right.png" alt="Vehicle turning right." width="500">

The -x distance is colored in the following image. This distance will be the input for the PID control loop, also known as the error. The PID control will attempt to minimize this error to zero, thus centering the vehicle on the lanes.
<img src="tracking-right-distance.png" alt="Error" width="500">

### Conclusions

The development of the lane keeping system consisted on the successful development of its core components measured at the beginning of this project. One of the biggest struggles for this project where the lane tracking and control systems. A naive approach was taken at first, which consisted on calculating the intersection point between the detected lanes and measuring the deviation from the image center. This deviation would be the input to the PID and must be minimized to zero. Nonetheless, this proved to result unsuccessful. When the vehicle is turning the center of the image moves slightly to the turning side and the vehicle slowly starts to lean towards this edge until it completely goes over the lane. 

Therefore, a more complex algorithm was proposed. For this, data and video was recovered from manual control tests and it was found that the detected lanes tend to follow a normal distribution. This meant that they have a mean value and that they could be filtered via a Kalman Filter. Further analysis proved that by trying to keep the vehicle centered on the mean values for the lanes would also mean that the vehicle would remain centered.

Implementing this algorithm required going a step further than what was though in this course. A Kalman Filter was a key aspect for this project to work. It’s such a powerful filter with a lot of versatility. It allowed to filter outliers and produced a smooth lane detection.

A limiting factor for the performance of this vehicle was the physical construction. The vehicle is a two-wheeled robot with a caster wheel on front. By having this construction type, any debris found on the track caused the vehicle to jump and turn at the same time. This made it harder to control the vehicle and limited the speed at which the vehicle could be driven. The maximum speed for reliable results was 40%. At this speed the vehicle was capable of driving around the track with no errors.

As a recommendation, a four wheeled robot would be a better design choice for this kind of vehicle. Finally, a more complex algorithm for detecting lanes would be desirable. By estimating a second degree polynomial and the lane curvature, the vehicle can be better centered. However, doing so would result in a completely different implementation to what was thought throughout this course. 

All in all, I believe what was achieved with the knowledge learned in this course and the implementation of a Kalman Filter produced great results. This is the last class in my bachelor’s degree, and it has allowed me to practice everything that I have learned on previous semesters.

### References

_I hereby affirm that I have done this activity with academic integrity._